In [1]:
import pandas as pd
import numpy as np
from sklearn.multioutput import RegressorChain
from sklearn.metrics import mean_squared_error as mse

In [2]:
# to import data sets for each level
# is in order to get a clear structure of matrix
eva_1 = pd.read_pickle('eva_l0.pkl')
eva_2 = pd.read_pickle('eva_l1.pkl')
eva_3 = pd.read_pickle('eva_l2.pkl')
eva_4 = pd.read_pickle('eva_l3.pkl')
eva_list = [eva_1,eva_2,eva_3,eva_4]
# get the train&test sets
x_train = pd.read_pickle('x_train.pkl')
x_test = pd.read_pickle('x_test.pkl')
y_train = pd.read_pickle('y_train.pkl')
y_test = pd.read_pickle('y_test.pkl')

In [3]:
def get_lgb():
    from lightgbm import LGBMRegressor as lgbr
    return lgbr(
        n_jobs = -1,
        boosting_type= 'gbdt',
        objective = "regression",
        metric = "rmse",
        num_leaves =64,
        learning_rate = 0.1, #0.1
        # 0.05 - 74~75
        #bagging_fraction = 0.7,
        #feature_fraction = 0.5,
        #bagging_frequency = 5,
        #bagging_seed = 42,
        verbosity = 1,
        #seed= 42,
        force_col_wise=True)
l1 = RegressorChain(get_lgb())
l1.fit(x_train, y_train)

[LightGBM] [Info] Total Bins 15820
[LightGBM] [Info] Number of data points in the train set: 4026248, number of used features: 67
[LightGBM] [Info] Start training from score 4.943179
[LightGBM] [Info] Total Bins 16075
[LightGBM] [Info] Number of data points in the train set: 4026248, number of used features: 68
[LightGBM] [Info] Start training from score 4.944538
[LightGBM] [Info] Total Bins 16330
[LightGBM] [Info] Number of data points in the train set: 4026248, number of used features: 69
[LightGBM] [Info] Start training from score 4.945578
[LightGBM] [Info] Total Bins 16585
[LightGBM] [Info] Number of data points in the train set: 4026248, number of used features: 70
[LightGBM] [Info] Start training from score 4.946270
[LightGBM] [Info] Total Bins 16840
[LightGBM] [Info] Number of data points in the train set: 4026248, number of used features: 71
[LightGBM] [Info] Start training from score 4.947360
[LightGBM] [Info] Total Bins 17095
[LightGBM] [Info] Number of data points in the tra

RegressorChain(base_estimator=LGBMRegressor(force_col_wise=True, metric='rmse',
                                            n_jobs=-1, num_leaves=64,
                                            objective='regression',
                                            verbosity=1))

In [4]:
# get future 28 days base forecast
train_pred = l1.predict(x_train)
test_pred = l1.predict(x_test)

In [5]:
# define the function for calculating MASE
# the epsilon represets the default value for the situation that the denominator is 0
from sklearn.metrics import mean_absolute_error
def forecast_mase(forecast_1, truth_1, epsilon=1):
    mase = []
    forecast = np.array(forecast_1)
    truth = np.array(truth_1)
    tru_diff = np.abs(truth[:, 1:] - truth[:, :-1])  # achieve .diff()
    mean_diff = np.mean(tru_diff, axis=1)
    for i in range(len(forecast)):
        mae_ = mean_absolute_error(forecast[i, :], truth[i, :])
        mean_diff_i = mean_diff[i] if mean_diff[i] != 0 else epsilon
        mase.append(mae_ / mean_diff_i)
    return mase

In [6]:
# construct SUMMING MATRIX for BU, TD, AND MINT 
# total of 4 levels
def SMatrix(level_number, bottom_level_number, eva_list_ts):
    l = []
    i = 0
    while i <= level_number:
        if i == 0:  # （1，2160）
            # for top level
            a = np.ones(bottom_level_number).reshape(1,2160)
            l.append(a)
            
        elif i == 1:  #（3，2160）
            # for l2
            l.append([])
            o = [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]]
            for x1 in range(len(eva_list_ts[i]['state_id'].unique())):
                r = (o[x1] * (bottom_level_number // len(o) + 1))[:bottom_level_number]
                l[i].append(r)
        elif i == 2:  #（10，2160）
            # for l3 store
            l.append([])
            for x2 in range(len(eva_list_ts[i]['store_id'].unique())):
                o = np.zeros(bottom_level_number)
                o[x2::9] = 1
                l[i].append(o)
        elif i == 3:  # （2160，2160）
            # for l4
            l.append(np.eye(bottom_level_number))
        else:
            break
        i += 1
    S = np.concatenate((l[0], l[1], l[2], l[3]), axis=0)
    return S

In [7]:
S_Matrix = SMatrix(4,2160, eva_list)

In [8]:
################# BU method:
# G matrix for BU
g1 = np.zeros((2160,14))#（ bottom_level , except_bottom level）
g2 = np.eye(2160)# bottom level
G_bu = np.concatenate((g1,g2), axis=1)

################# TD method:
# G matrix for TD
def G_Matrix_TD(bottom_level, top_level, ts_length, col_bo, col_to, ts_amount):
    from typing import Optional
    from tqdm import tqdm
    prop_mean = []
    n = int(len(bottom_level[col_bo])/ts_length)
    for i in tqdm(range(n)):
        prop = []
        #prop = np.divide(np.array(bottom_level[col_bo][i*ts_length : (i+1)*ts_length]), np.array(top_level[col_to]))
        #, out=np.zeros_like(vector1), where=(vector2 != 0)
        with np.errstate(divide='ignore', invalid='ignore'):
            prop =  (bottom_level[col_bo].iloc[i*ts_length : (i+1)*ts_length].values/top_level[col_to].values)
        prop_mean.append(prop)
    prop_mean = np.nan_to_num(prop_mean)
    prop_mean = np.mean(prop_mean,axis = 1).reshape(-1,1)
    g2 = np.zeros((n, ts_amount-1))
    G = np.concatenate((prop_mean, g2), axis=1)
    return G

################# MinT method:
# G matrix for MinT
# generate the W estimate for constructing G
def W_matrix_G(forecast_1, truth_1):
    forecast = np.array(forecast_1)
    truth = np.array(truth_1)
    residuals_ = []
    for i in range(int(len(forecast))):
        res = np.mean(np.dot(truth[i,:]-forecast[i,:],truth[i,:]-forecast[i,:]))
        residuals_.append(res)
    W = np.diag(residuals_)
    return W, residuals_

# parameters: bottom level data, top level data, time series length, 
G_td = G_Matrix_TD(eva_list[3], eva_list[0], 1941, 'sales', 'sales', 2174) 
# based on the book Forecasting:principle&practice chapter11
W , residuals =W_matrix_G(test_pred, y_test.values)
G_mint = np.linalg.inv(S_Matrix.T@np.linalg.inv(W)@S_Matrix)@S_Matrix.T@np.linalg.inv(W)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2160/2160 [00:00<00:00, 4011.04it/s]


In [9]:
SG_bu = np.dot(S_Matrix, G_bu)
SG_td = np.dot(S_Matrix, G_td)
SG_mint = np.dot(S_Matrix,G_mint)

In [12]:
y_base_frec = test_pred
y_coh_frec_bu = np.dot(SG_bu, y_base_frec)
y_coh_frec_td = np.dot(SG_td, y_base_frec)
y_coh_frec_mint = np.dot(SG_mint, y_base_frec)

In [13]:
# get mase series of base global lgbm(including MASE values for each time series)
re_base = forecast_mase(y_base_frec, y_test)
re_bu = forecast_mase(y_coh_frec_bu, y_test)
re_td = forecast_mase(y_coh_frec_td, y_test)
re_mint = forecast_mase(y_coh_frec_mint, y_test)

In [14]:
# MASE for the whole model
print("lgbm's forecasts:", np.mean(re_base))
print("lgbm with BU's forecasts:", np.mean(re_bu))
print("lgbm with TD's forecasts:", np.mean(re_td))
print("lgbm with MinT's forecasts:", np.mean(re_mint))

lgbm's forecasts: 1.3135465788316951
lgbm with BU's forecasts: 1.3161529467197781
lgbm with TD's forecasts: 1.0561110957215272
lgbm with MinT's forecasts: 1.3159135970614464
